In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [44]:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [45]:
#  nltk.download('wordnet', '/root/nltk_data')

In [46]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [47]:
df = pd.read_csv('/kaggle/input/plant-disease-coversation/Preprocessed_Coversational_Data.csv')

df.drop(['Unnamed: 0'], axis = 1, inplace=True)

df.head()

,User Question,Identified Issue,Symptoms,Management Steps,Response,Preprocessed_Question,Preprocessed_Identified_Issue,Preprocessed_Symptoms,Preprocessed_Mng_Steps,Rreprocessed_Response,User_Question_Length,Response_Length,Ngrams
0,How can I save my plant if it has powdery mild...,"Powdery Mildew,",A gray or white powdery growth on the surface ...,growers should avoid getting the leaves wet wh...,"Hello, plant parent! 🌱 Your plant might have p...",save plant powdery mildew,powdery mildew,gray white powdery growth surface leaf stem,grower avoid getting leaf wet watering plant d...,hello plant parent plant might powdery mildew ...,11,91,"['save plant', 'plant powdery', 'powdery milde..."
1,I have black spots on my roses and the leaves ...,Black Spot,"Small, round, black spots on the leaves; leave...",Remove and destroy affected leaves.<br> Water ...,"Hello, rose gardener! 🌹 It looks like your ros...",black spot rose leaf starting turn yellow fall...,black spot,small round black spot leaf leaf may turn yell...,remove destroy affected leaf water base plant ...,hello rose gardener look like rose dealing bla...,27,169,"['black spot', 'spot rose', 'rose leaf', 'leaf..."
2,I have yellow spots on my plant leaves. What's...,"Bacterial Leaf Spot,",Irregularly shaped yellow spots that turn brow...,Remove and destroy affected leaves.<br> Avoid ...,"Hello, worried gardener! 🌱 It sounds like your...",yellow spot plant leaf causing,bacterial leaf spot,irregularly shaped yellow spot turn brown blac...,remove destroy affected leaf avoid overhead wa...,hello worried gardener sound like plant might ...,18,89,"['yellow spot', 'spot plant', 'plant leaf', 'l..."
3,I have strange black spots on my plant leaves....,Black Spot Disease,"Black spots on the leaves, often with a yellow...",Remove and destroy affected leaves.\n\n Avoid ...,"Hello, gardener! It sounds like your plant is ...",strange black spot plant leaf problem fix,black spot disease,black spot leaf often yellow halo spot may eve...,remove destroy affected leaf avoid overhead wa...,hello gardener sound like plant suffering blac...,19,31,"['strange black', 'black spot', 'spot plant', ..."
4,The tips of my plant's leaves are turning brow...,"Tip burn,",Brown or black tips on leaves that are also cu...,Adjust the pH of your plant's soil.<br> Make s...,Curling leaves with brown or black tips can be...,tip plant leaf turning brown curling know migh...,tip burn,brown black tip leaf also curled,adjust ph plant soil make sure plant getting e...,curling leaf brown black tip sign tip burn oft...,19,75,"['tip plant', 'plant leaf', 'leaf turning', 't..."


In [48]:
data = df['User Question'] +' '+ df['Response']

In [49]:
#convert text into lower case
def lower_case(text):
    return text.lower()


# remove html tags
def remove_html_tags(text):
    cleaned_text = text.replace("<br>", " ")
    return cleaned_text

# remove emojis from given text specially from chatbot response.
def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

# tokenize the given text
def tokenize_sentences(text):
    """Tokenize each sentence."""
    return word_tokenize(text)

# remove stopwords and punctuation from text
def remove_stopwords_punctuations(words):
    """Remove stopwords, punctuation, and special characters."""
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalnum() and word not in string.punctuation]
    return ' '.join(filtered_words)

# # lemmatizing the text string
# def perform_lemmatization(words):
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
#     return ' '.join(lemmatized_words)

In [50]:
## Text Wrangling Piepline!!!

def cleaning_text(text):
    text = lower_case(text)
    text = remove_html_tags(text)
    text = remove_emoji(text)
    text = tokenize_sentences(text)
    text = remove_stopwords_punctuations(text)
#     text = perform_lemmatization(text)
    return text

In [51]:
data = data.apply(lambda x: cleaning_text(x))

In [52]:
text_data = list(data.sample(2000))

In [53]:
text_data[:2]

['i have a leaf on my houseplant and it losing its color what wrong with it and how can i fix it it looks like your houseplant is suffering from iron chlorosis which is characterized by interveinal chlorosis yellowing of the tissue between the veins which remain green and stunted growth you can apply a chelated iron fertilizer according to package directions take a sample of the plant to your local cooperative extension office for diagnosis',
 'i found this on my plant what is it and how do i get rid of it hi there from the looks of it your plant is suffering from scale here are some ways you can fight back prune off any leaves or branches that are affected apply an insecticide to the plant following the directions on the label encourage natural predators like ladybugs or lacewings to help control the scale population with these steps your plant should start to recover']

In [54]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1  # Add 1 for the '0' padding


In [55]:
total_words

3156

In [56]:
# Create sequences of tokens
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Create predictors and labels
xs = input_sequences[:, :-1]
ys = tf.keras.utils.to_categorical(input_sequences[:, -1], num_classes=total_words)


In [57]:
# Build the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 236, 100)          315600    
                                                                 
 lstm_2 (LSTM)               (None, 150)               150600    
                                                                 
 dense_2 (Dense)             (None, 3156)              476556    
                                                                 
Total params: 942756 (3.60 MB)
Trainable params: 942756 (3.60 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
# Train the model
batch_size = 100  # Try reducing the batch size
# history = model.fit(xs, ys, epochs=10, verbose=1)
history = model.fit(xs, ys, epochs=100, batch_size=batch_size, verbose=1)

Epoch 1/100
2111/2111 [==============================] - 86s 40ms/step - loss: 4.1209 - accuracy: 0.2699
Epoch 2/100
2111/2111 [==============================] - 43s 20ms/step - loss: 2.7017 - accuracy: 0.4622
Epoch 3/100
2111/2111 [==============================] - 44s 21ms/step - loss: 2.3012 - accuracy: 0.5159
Epoch 4/100
2111/2111 [==============================] - 41s 20ms/step - loss: 2.0849 - accuracy: 0.5443
Epoch 5/100
2111/2111 [==============================] - 42s 20ms/step - loss: 1.9379 - accuracy: 0.5629
Epoch 6/100
2111/2111 [==============================] - 41s 19ms/step - loss: 1.8285 - accuracy: 0.5766
Epoch 7/100
2111/2111 [==============================] - 41s 19ms/step - loss: 1.7409 - accuracy: 0.5883
Epoch 8/100
2111/2111 [==============================] - 41s 20ms/step - loss: 1.6692 - accuracy: 0.5989
Epoch 9/100
2111/2111 [==============================] - 41s 19ms/step - loss: 1.6078 - accuracy: 0.6085
Epoch 10/100
2111/2111 [==============================]

In [67]:
num_words = 100

# Start the conversation loop
while True:
    # Get user input for the conversation
    user_input = input("You: ")
    
    # Check if the user wants to exit the conversation
    if user_input.lower() == 'exit':
        print("Exiting conversation...")
        break

    # Initialize seed_text with the user's input
    seed_text = user_input
    
    # Initialize generated_text to store the complete generated response
    generated_text = seed_text

    # Generate text based on user input
    for _ in range(num_words):
        # Tokenize the seed_text and pad the sequences
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        
        # Predict the next word
        predicted = np.argmax(model.predict(token_list), axis=-1)
        
        # Retrieve the predicted word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        
        # Update seed_text and generated_text
        seed_text += " " + output_word
        generated_text += " " + output_word

    # Output the generated response
    print("Bot:", generated_text)


You:  i have black spots on my roses and the leaves are falling off what is wrong and how do i fix it


1/1 [==============================] - 0s 20ms/step
Bot: i have black spots on my roses and the leaves are falling off what is wrong and how do i fix it hello rose gardener your roses might have black spot which is characterized by black spots on the leaves often with a yellow halo if left untreated the leaves will eventually turn yellow and fall off here are some management steps you can take remove and destroy infected leaves this will help stop the spread of the disease apply a fungicide labeled for black spot following the instructions on the label this will help to kill the fungus and protect your plant maintain proper plant spacing for air circulation this will help to reduce the amount of moisture on the leaves


You:  exit


Exiting conversation...


In [60]:
model.save('text-generator.h5')

In [61]:
model.save('Plant-Disease-Chatbot.h5')

In [66]:
data[17]

'i have black spots on my roses and the leaves are falling off what is wrong and how do i fix it hello rose gardener your roses may have black spot which is identified by small round black spots on the leaves these spots may be surrounded by a yellow halo and eventually the leaves turn yellow and drop off here are some tips to get rid of black spot cleanup duty remove and destroy infected leaves fungicide friend apply a fungicide specifically designed to kill black spot base watering water roses at the base of the plant early in the day so that the foliage dries quickly dodging water avoid getting water on the leaves when watering pruning and thinning prune out infected canes and thin the plant to increase air circulation mulch it up apply a thick layer of mulch around the base of the plant with these steps your roses will be free in no time'